# Pump.fun Tools

In [ ]:
from pumpfun import PumpFunWalletManager

In [ ]:
""" async with PumpFunWalletManager() as wallet_manager:
    wallet_data = await wallet_manager.create_multiple_wallets(5, "Wallet") """

In [ ]:
from pumpfun.pumpfun_trade_analyzer import PumpFunTradeAnalyzer
from icecream import ic

In [ ]:
async with PumpFunTradeAnalyzer() as parser:
    trade_info = await parser.analyze_transaction_by_signature("2mrPeUBHWsRJdEChNTUQWyQLxAPHMHGuwNGMDhFMecL5bfmfabCLGqBcEH6iVuMKRzdYhbsgVusFZC5aDB4xjYZH")
    if trade_info:
        print(trade_info)  # Muestra toda la información detallada
        # O obtén como diccionario
        trade_dict = trade_info.to_dict()